<a href="https://colab.research.google.com/github/gunavardhangolagani/AZURE-AI-VECTOR-SEARCH/blob/main/Review_Video_Recordings_for_Meeting_summary_and_Action_items.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Whisper setup

In [7]:
%%capture
!pip install openai-whisper

In [8]:
import whisper
model = whisper.load_model("base")
print(model)

Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(80, 512, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0-5): 6 x ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=512, out_features=512, bias=True)
          (key): Linear(in_features=512, out_features=512, bias=False)
          (value): Linear(in_features=512, out_features=512, bias=True)
          (out): Linear(in_features=512, out_features=512, bias=True)
        )
        (attn_ln): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=512, out_features=2048, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=2048, out_features=512, bias=True)
        )
        (mlp_ln): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_post): LayerNorm((512,), eps=1e-05,

In [9]:
%%capture
! pip install ffmpeg-python pytube

## Basic requirements

In [10]:
%%capture
!pip install streamlit
#openai
!pip install langchain
!pip install langchain-community
!pip install python-dotenv
!pip install langchain-fireworks
!pip install requests
!pip install fireworks-ai
!pip install pytube
!pip install langchain-community

## Fireworks(LlaMa-3)Setup

In [11]:
import os
import requests
import fireworks.client
import whisper
import subprocess
from fireworks.client import Fireworks
from pytube import YouTube
# import openai
#load_dotenv()
#openai_api_key = os.getenv("OPENAI_API_KEY")
#fireworks_api_key = os.getenv("FIREWORKS_API_KEY")

In [12]:
import getpass
api_key = getpass.getpass("Enter your API Key...")

Enter your API Key...··········


In [13]:
# Function to download YouTube video
def download_youtube_video(youtube_url, video_output):
    yt = YouTube(youtube_url)
    video_stream = yt.streams.filter(only_audio=True).first()
    video_stream.download(filename=video_output)
# Function to extract audio from video
def extract_audio(video_path, audio_output):
    command = f"ffmpeg -i {video_path} -q:a 0 -map a {audio_output}"
    subprocess.run(command, shell=True)

# Function to transcribe audio using Whisper
def transcribe_audio(audio_path):
    model = whisper.load_model("base")
    result = model.transcribe(audio_path)
    return result["text"]

# Function to extract action items using Fireworks
def extract_action_items(transcription):
    prompt = f"Extract action items from the following meeting transcript:\n\n{transcription}\n\nAction items:"
    fireworks.client.api_key = api_key
    client = Fireworks(api_key=api_key)
    fw_model_dir = "accounts/fireworks/models/"
    model = "llama-v3-70b-instruct"
    model = fw_model_dir + model
    response = fireworks.client.Completion.create(
        model=model,
        prompt=prompt,
        max_tokens=256,
        temperature=0
    )
    action_items = response.choices[0].text
    return action_items

# Main process function
def process_video(video_file=None, youtube_url=None):
    video_file_path = "downloaded_video.mp4"
    audio_file_path = "output_audio.mp3"

    if youtube_url:
        download_youtube_video(youtube_url, video_file_path)
    elif video_file:
        with open(video_file_path, "wb") as f:
            f.write(video_file.read())

    extract_audio(video_file_path, audio_file_path)
    transcription = transcribe_audio(audio_file_path)
    action_items = extract_action_items(transcription)

    os.remove(video_file_path)  # Clean up video file
    os.remove(audio_file_path)  # Clean up audio file

    return transcription, action_items

# Input from user
youtube_url = input("Enter YouTube URL (leave blank if uploading a video file): ").strip()
video_file = None

if not youtube_url:
    from IPython.display import display
    import ipywidgets as widgets

    video_file = widgets.FileUpload(accept=".mp4", multiple=False)
    display(video_file)

    while not video_file.value:
        pass  # Wait for the user to upload a file

    video_file = next(iter(video_file.value.values()))

# Process the video
transcription, action_items = process_video(video_file=video_file, youtube_url=youtube_url)

# Display the results
import textwrap
wrapped_transcription = "\n".join(textwrap.wrap(transcription, width=80))
print("### Transcription:\n")
print(wrapped_transcription)
print("\n### Action Items:\n")
print(action_items)

Enter YouTube URL (leave blank if uploading a video file): https://www.youtube.com/watch?v=rrLOkkIGGD0&pp=ygUnR3JvdXAgZGlzY3Vzc2lvbiB0YWxrcyB0b3AgdW5pdmVyc2l0aWVz


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


### Transcription:

 Justin Trump Before we start this debate, there have been lots of points of
view on whether to allow 100% of the ARR in money brand and whether it should be
in the health economy. See, we are 25 of us and we have got about 30 minutes.
30-35 minutes. So please restrict your sense to at least max five sentences at
each time, you speak, and also allow maintain discipline by allowing your fellow
colleague to speak. And that's it. And you must have had a lot of group
discussions in your college so you know it better than us. So we open the house,
you can begin. I am against this particular behind retail. I believe because the
first reason is it is going to display a lot of retail moment pop stores which
an argument against this is given that it will create jobs. But the kind of
nature of the job that will create will be mostly blue collars and everywhere
the jobs while the people that they will display would be the self-owned
entrepreneurs would be tensile owning a stor

### Gradio interface

In [15]:
%%capture
!pip install gradio

In [16]:
import gradio as gr

def process_video(video_file=None, youtube_url=None):
    video_file_path = "downloaded_video.mp4"
    audio_file_path = "output_audio.mp3"

    if youtube_url:
        download_youtube_video(youtube_url, video_file_path)
    elif video_file:
        with open(video_file_path, "wb") as f:
            f.write(video_file.read())

    extract_audio(video_file_path, audio_file_path)
    transcription = transcribe_audio(audio_file_path)
    action_items = extract_action_items(transcription)

    os.remove(video_file_path)  # Clean up video file
    os.remove(audio_file_path)  # Clean up audio file

    return transcription, action_items

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# Meeting Summary and Action Items Extractor")

    with gr.Tab("Upload Video"):
        with gr.Row():
            video_file_input = gr.File(label="Upload Video", file_types=["video"])

    with gr.Tab("YouTube URL"):
        with gr.Row():
            youtube_url_input = gr.Textbox(label="Enter YouTube URL")

    extract_button = gr.Button("Extract Audio and Transcribe")

    transcription_output = gr.Textbox(label="Transcription", lines=10)
    action_items_output = gr.Textbox(label="Action Items", lines=5)

    extract_button.click(
        process_video,
        inputs=[video_file_input, youtube_url_input],
        outputs=[transcription_output, action_items_output]
    )

demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://00ccb68ad440e7fff2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
